# Modelo DDS

Modelado y comparación de la generación de señales con DDS Compiler.

Importo los modulos necesarios

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def fpga_data():


    s       = np.loadtxt('../files/simple.txt')
    N       = int(s.size / 2)
    s_new   = np.reshape(s, (N, 2))
    x_cmpx  = s_new[:, 0] + 1j*s_new[:, 1]
    XF      = np.fft.fft(x_cmpx, axis=0, norm=None)
    f_axis  = np.linspace(0, 100, N)

    plt.figure(figsize=(10,8))
    plt.subplot(221)
    plt.title("sin and cos: time")
    plt.plot(np.real(x_cmpx), '.-r', label='real')
    plt.plot(np.imag(x_cmpx), '.-b', label='imag')
    plt.xlabel("time, bins")
    plt.grid()
    plt.legend()

    plt.subplot(223)
    plt.plot(f_axis, np.abs(XF), '.-b')
    plt.title("sin and cos: frequency")
    plt.xlabel("freq, MHz")
    plt.grid()
    s       = np.loadtxt('../files/psk.txt')
    N       = int(s.size / 2)
    s_new   = np.reshape(s, (N, 2))
    x_cmpx  = s_new[:, 0] + 1j*s_new[:, 1]
    XF      = np.fft.fft(x_cmpx, axis=0, norm=None)
    f_axis  = np.linspace(0, 100, N)

    plt.subplot(222)
    plt.title("PSK: time")
    plt.plot(np.real(x_cmpx), '.-r', label='real')
    plt.plot(np.imag(x_cmpx), '.-b', label='imag')
    plt.xlabel("time, bins")
    plt.grid()
    plt.legend()

    plt.subplot(224)
    plt.plot(f_axis, np.abs(XF), '.-b')
    plt.xlabel("freq, MHz")
    plt.title("PSK: frequency")
    plt.grid()

    plt.tight_layout()


    s       = np.loadtxt('../files/fsk.txt')
    N       = int(s.size / 2)
    s_new   = np.reshape(s, (N, 2))
    x_cmpx  = s_new[:, 0] + 1j*s_new[:, 1]
    XF      = np.fft.fft(x_cmpx, axis=0, norm=None)
    f_axis  = np.linspace(0, 100, N)

    plt.figure(figsize=(10,8))
    plt.subplot(221)
    plt.title("FSK: time")
    plt.plot(np.real(x_cmpx), '.-r')
    plt.plot(np.imag(x_cmpx), '.-b')
    plt.xlabel("time, bins")
    plt.grid()

    plt.subplot(223)
    plt.plot(f_axis, np.abs(XF), '.-b')
    plt.title("FSK: frequency")
    plt.xlabel("freq, MHz")
    plt.grid()


    s       = np.loadtxt('../files/lfm.txt')
    N       = int(s.size / 2)
    s_new   = np.reshape(s, (N, 2))
    x_cmpx  = s_new[:, 0] + 1j*s_new[:, 1]
    XF      = np.fft.fft(x_cmpx, axis=0, norm=None)
    f_axis  = np.linspace(0, 100, N)

    plt.subplot(222)
    plt.title("LFM: time")
    plt.plot(np.real(x_cmpx), '.-r')
    plt.plot(np.imag(x_cmpx), '.-b')
    plt.xlabel("time, bins")
    plt.grid()

    plt.subplot(224)
    plt.plot(f_axis, np.abs(XF), '.-b')
    plt.xlabel("freq, MHz")
    plt.title("LFM: frequency")
    plt.grid()

    plt.tight_layout()
    
    return 0

In [ ]:
def dds_lut():

    sinus_l = np.sin(2*np.pi*np.linspace(0, 1, 2**16))

    # phase increment
    incrm_t = 4
    incrm_k = 20


    sinus_t = np.zeros(2**16)
    sinus_k = np.zeros(2**16)

    phase_l = np.zeros(2**16)
    phase_t = np.zeros(2**16)
    phase_k = np.zeros(2**16)
    for n in range(0, 2**16):

        phase_l[n] = n

        phase_t[n] = (incrm_t*n) % 2**16
        sinus_t[n] = sinus_l[(incrm_t*n) % 2**16]

        phase_k[n] = ((n// 50)**2) % 2**16
        sinus_k[n] = sinus_l[((n // 50)**2) % 2**16]

    plt.figure(figsize=(10,8))

    plt.subplot(311)
    plt.title('DDS output, phase increment is {}'.format(1))
    plt.plot(sinus_l, '.-b', label='signal')
    plt.plot(phase_l/2**16, '.-g', label='phase')
    plt.grid()
    plt.legend()

    plt.subplot(312)
    plt.title('DDS output, phase increment is {}'.format(incrm_t))
    plt.plot(sinus_t, '.-b', label='signal')
    plt.plot(phase_t/2**16, '.-g', label='phase')
    plt.grid()
    plt.legend()

    plt.subplot(313)
    plt.title('DDS output for LFM')
    plt.plot(sinus_k, '.-b', label='signal')
    plt.plot(phase_k/2**16, '.-g', label='phase')
    plt.grid()
    plt.legend()

    plt.tight_layout()
    
    return 0

In [ ]:
def psk(s, n_clocks):
    N = int(s.size)
    tmp = 1
    psi = np.zeros((N, 1))
    for i in range(N):
        if(np.mod(i , n_clocks) == 0):
            tmp = ~(tmp)
        if(tmp == 1):    
            s[i] = 1 * s[i]
        else:
            s[i] = -1 * s[i]
        psi[i] = tmp
    return s, psi

In [ ]:
# Function for phase increment calculation from desirable frequency
# Here assuming sample clock is Fclk in MHz
# des_freq is the desired frequency
# Btheta is the number of bits in the phase accumulator

def phase_incr(Fclk, des_freq, Btheta):
    return int((des_freq * 2**Btheta)/Fclk)

def dds_freq(Fclk,phase_incr,Btheta):
    return (Fclk*phase_incr)/2**Btheta

In [ ]:
dds_lut()

In [ ]:
fpga_data()

# Verificación
Lo que sigue es una comprobación de que el diseño con el filtro portotipo es lo mismo que con los $h_i$. 

Para hacer la comprobación genero señales de referencia y después las filtro con los dos esquemas de filtrado, con el prototipo y con el FIR multietapas.

In [ ]:
q           = 100       # resampling factor (decimation/interpolation)
fs          = 250.0e6   # sample rate, Hz
T           = 30e-6     # length of signal, sec
F1          = 0.5e6       # start frequency, Hz
F2          = 5e6       # end frequency, Hz
N           = int(fs*T) # number of signal's samples

In [ ]:
print(phase_incr(250e6,34.56e6,16))
print(dds_freq(250e6,9059,16))

In [ ]:
fname_real = '../files/dds_real_out.txt'
fname_imag = '../files/dds_imag_out.txt'
real_out   = np.loadtxt(fname_real)
imag_out   = np.loadtxt(fname_imag)
n           = np.size(real_out)
t           = np.linspace(0, n/fs, n, endpoint=False)
print(real_out.size)

In [ ]:
yf = np.fft.fft(real_out)
xf = np.fft.fftfreq(n, 1/fs)

In [ ]:
plt.figure(figsize=(10,5))
#plt.suptitle("Comparación de resultados: 1 etapa vs 3 etapas",fontsize=16,fontweight='bold')

#plt.subplot(221)
plt.plot(t/1e-6,real_out, 'o-b', label="real")
plt.plot(t/1e-6,imag_out, 'o-r', label="imag")
plt.title('time domain')
plt.legend(loc="upper left")
plt.xlabel('t, usec')
plt.grid()

In [ ]:
fig, ax = plt.subplots()
ax.plot(xf[:int(n/2)], 2.0/n * np.abs(yf[0:int(n/2)]))
#ax.plot(xf[:int(N/2)], 1.0/N * np.abs(yf[0:int(N/2)]))
#ax.plot(xf[:int(N/2)], np.abs(yf[0:int(N/2)])/N)
ax.set_xlim(0,fs/2)
plt.xlabel('f, MHz (sample rate is {:.2e} MHz)'.format(fs))
plt.title('frequency domain')
plt.grid()
print(xf[np.argmax(2.0/n*np.abs(yf[0:int(n/2)]))])
print(dds_freq(250e6,9059,16))

In [ ]:
xr_fft  = np.fft.fftshift(np.fft.fft(real_out, n=n, axis=-1))
freqs = np.fft.fftshift(np.fft.fftfreq(n, 1/fs))

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(freqs/1e6, np.abs(xr_fft), '.-r', label="Py decimated")
#plt.plot(freqs/1e6, np.abs(xdec_fft_et3_fir_out)/2**9, '.-b', label="FIR decimated")
plt.xlabel('f, MHz (sample rate is {:.2e} MHz)'.format(fs))
plt.title('frequency domain')
plt.grid()
plt.legend(loc="upper left")